<a href="https://colab.research.google.com/github/Ansh191/COVID19-Analysis/blob/master/NN_VS_LMFIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from google.colab import drive
import requests
from bs4 import BeautifulSoup
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
cases = 'https://covid.ourworldindata.org/data/ecdc/total_cases.csv'
df = pd.read_csv(cases)
df = df.fillna(0)
del df['date']
df = df.astype('int64')
lmfit = pd.read_csv('/gdrive/My Drive/COVID19/Predictions_lmfit_98.csv')
lmfit.rename({'Unnamed: 0': "Day"}, axis='columns', inplace=True)
lmfit.set_index('Day', inplace=True)
lmfit = lmfit.astype('int64')
nn = pd.read_csv('/gdrive/My Drive/COVID19/Predictions_nn_98.csv')
nn.rename({"Unnamed: 0": "Day"}, axis='columns', inplace=True)
nn.set_index('Day', inplace=True)

In [5]:
if df.iloc[len(df.index) - 1].isnull().any():
  print("Already exists. Deleting...")
  df.drop(df.tail(1).index, inplace=True)
tmp = pd.DataFrame(columns=df.columns, index=[len(df.index)], dtype='int64')
for c in df.columns[1:]:
  c2 = c
  if df[c].iloc[-1] > 1000:
    c = "us" if c == 'United States' else c
    c = "uk" if c == 'United Kingdom' else c
    c = c.replace(' ', '-').lower()
    page = requests.get(f'https://www.worldometers.info/coronavirus/country/{c}/')
    soup = BeautifulSoup(page.content, 'html.parser')
    current = int(soup.find(class_='maincounter-number').find('span').text.replace(',', ''))
    if current > df[c2].iloc[-1]:
      print(f"{c}: {current} {df[c2].iloc[-1]}")
      tmp[c2].iloc[0] = current
  
if not (df.index == len(df.index)).any():
  df = pd.concat([df, tmp])
else:
  df.loc[len(df.index)] = tmp.values.flatten()

algeria: 1468 1423
australia: 5919 5844
austria: 12635 12297
belgium: 22194 20814
brazil: 13717 12056
canada: 17847 16653
chile: 5116 4815
croatia: 1282 1222
czech-republic: 4944 4822
denmark: 5071 4681
dominican-republic: 1956 1828
egypt: 1450 1073
estonia: 1149 1108
finland: 2308 2176
france: 109069 74390
germany: 107458 99225
greece: 1832 1755
iceland: 1586 1562
india: 5311 4421
indonesia: 2738 2491
iran: 62589 60500
iraq: 1122 1031
ireland: 5709 5364
israel: 9248 8904
italy: 135586 132547
japan: 3906 3817
luxembourg: 2970 2843
malaysia: 3963 3793
morocco: 1184 1120
netherlands: 19580 18803
norway: 5903 5755
pakistan: 4009 3864
peru: 2954 2561
philippines: 3764 3660
poland: 4848 4413
portugal: 12442 11730
qatar: 2057 1832
romania: 4417 4057
russia: 7497 6343
saudi-arabia: 2795 2523
serbia: 2447 2200
singapore: 1481 1375
slovenia: 1059 1021
south-africa: 1749 1686
spain: 140617 135032
sweden: 7693 7206
switzerland: 22253 21574
turkey: 34109 30217
ukraine: 1462 1319
united-arab-emirat

In [6]:
country = 'United States'

fig = go.Figure()
fig.add_trace(go.Scatter(x=df[country].index.values, y=df[country].values, mode='markers', name='Data'))
fig.add_trace(go.Scatter(x=nn[country].index.values, y=nn[country].values, mode='markers+lines', name='Neural Network'))
fig.add_trace(go.Scatter(x=lmfit[country].index.values, y=lmfit[country].values, mode='markers+lines', name='Lmfit'))
fig.update_layout(
    title = {'text': f"Total Predicted Cases in {country}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'bottom'},
    xaxis_title = "Day",
    yaxis_title = "Total Cases",
    font = dict(
        family="Courier New, monospace",
        size=18
    ),
    xaxis=dict(range=[df[country][df[country] > 100].first_valid_index(), 125])
)
print(f"NN predicted cases:    {np.max(nn[country])}")
print(f"Lmfit predicted cases: {np.max(lmfit[country])}")
print(f"NN tomorrow total cases: {nn[country].iloc[0]}")
print(f"Lmfit tomorrow total cases: {lmfit[country].iloc[0]}")
fig.show()

NN predicted cases:    544448
Lmfit predicted cases: 523736
NN tomorrow total cases: 390887
Lmfit tomorrow total cases: 388969


In [7]:
nndiff = nn[nn[country].diff() > 0][country].diff().values
nndiff[0] = nn[country].iloc[0] - df[country].iloc[-2]

lmdiff = lmfit[country].diff().values
lmdiff[0] = lmfit[country].iloc[0] - df[country].iloc[-2]

fig = go.Figure()
fig.add_trace(go.Bar(x=df[df[country] > 100][country].diff().index.values,
                     y=df[df[country] > 100][country].diff().values,
                     text=df[df[country] > 100][country].diff().values,
                     textposition='auto', name='Data', width=0.9))
fig.add_trace(go.Bar(x=nn.index.values, y=nndiff, text=None, textposition='auto', name='Neural Network', width=0.9, opacity=0.75))
fig.add_trace(go.Bar(x=lmfit.index.values, y=lmdiff, text=None, textposition='auto', name='Lmfit', width=0.9, opacity=0.75))
fig.update_layout(
    title = {'text': f"New Cases Per Day in {country}", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'bottom'},
    xaxis_title = "Day",
    yaxis_title = "New Cases",
    font = dict(
        family="Courier New, monospace",
        size=18
    ),
    yaxis=dict(range=[0, max(np.max(nndiff), np.max(lmdiff), np.nanmax(df[df[country] > 100][country].diff().values)) + 1000]),
    barmode='overlay'
)